In [1]:
import numpy as np

In [2]:
def hex2bits(hex):
    return [int(char) for char in "{0:b}".format(int(hex, 16)).zfill((len(hex)-2)*4)]

def bits2hex(bits):
    bits_string = ''.join([str(x) for x in bits])
    return hex(int(bits_string, 2))[2:]

print(bits2hex([1,1,1,1,1,1,1,1]))

ff


In [3]:
initial_h0 = hex2bits('0x6a09e667')
initial_h1 = hex2bits('0xbb67ae85')
initial_h2 = hex2bits('0x3c6ef372')
initial_h3 = hex2bits('0xa54ff53a')
initial_h4 = hex2bits('0x510e527f')
initial_h5 = hex2bits('0x9b05688c')
initial_h6 = hex2bits('0x1f83d9ab')
initial_h7 = hex2bits('0x5be0cd19')

In [4]:
initial_k = [hex2bits(bits) for bits in [
   '0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5',
   '0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', '0x9bdc06a7', '0xc19bf174',
   '0xe49b69c1', '0xefbe4786', '0x0fc19dc6', '0x240ca1cc', '0x2de92c6f', '0x4a7484aa', '0x5cb0a9dc', '0x76f988da',
   '0x983e5152', '0xa831c66d', '0xb00327c8', '0xbf597fc7', '0xc6e00bf3', '0xd5a79147', '0x06ca6351', '0x14292967',
   '0x27b70a85', '0x2e1b2138', '0x4d2c6dfc', '0x53380d13', '0x650a7354', '0x766a0abb', '0x81c2c92e', '0x92722c85',
   '0xa2bfe8a1', '0xa81a664b', '0xc24b8b70', '0xc76c51a3', '0xd192e819', '0xd6990624', '0xf40e3585', '0x106aa070',
   '0x19a4c116', '0x1e376c08', '0x2748774c', '0x34b0bcb5', '0x391c0cb3', '0x4ed8aa4a', '0x5b9cca4f', '0x682e6ff3',
   '0x748f82ee', '0x78a5636f', '0x84c87814', '0x8cc70208', '0x90befffa', '0xa4506ceb', '0xbef9a3f7', '0xc67178f2' ]]

In [5]:
def printx(list, state):
    print()
    print("==> " + state + " <==")
    print("length list : " + str(len(list)))
    print(list)
    print()

In [6]:
def hex2bits(hex):
    return [int(char) for char in "{0:b}".format(int(hex, 16)).zfill((len(hex)-2)*4)]

def string2bits(string):
    ords = [bin(ord(char))[2:].zfill(8) for char in string]
    bits = [[int(char) for char in word] for word in ords]
    return np.array(bits).flatten()


In [7]:
"{0:b}".format(88).zfill(64)

'0000000000000000000000000000000000000000000000000000000001011000'

In [8]:
def preprocessing(message):
    message = string2bits(message).tolist()
    L = len(message)
    message.append(1)
    message = message + [0] * (512 - ((L + 1 + 64) % 512))
    message = message + [int(char) for char in "{0:b}".format(L).zfill(64)]
    
    return chunk(message) 

def chunk(message):
    return [message[i:i + 512] for i in range(0, len(message), 512)]

def create_message_schedule(chunk):
    w = [chunk[i:i + 32] for i in range(0, len(chunk), 32)]
    w = w + [[0] * 32] * 48
    for i in range(16, 64):
        s0 = bitsXOR(bitsXOR(rightrotate(w[i-15], 7), rightrotate(w[i-15], 18)), rightshift(w[i-15], 3))
        s1 = bitsXOR(bitsXOR(rightrotate(w[i-2], 17), rightrotate(w[i-2], 19)), rightshift(w[i-2], 10))
        w[i] = bitsADDs([w[i-16], s0, w[i-7], s1])
    return w
    
def rightrotate(bits, n):
    return bits[-n:] + bits[:-n]

def rightshift(bits, n):
    return [0] * n + bits[:-n]

def bitsXOR(bits1, bits2):
    xor_result = []
    for i in range(len(bits1)):
        if(bits1[i] != bits2[i]):
            xor_result.append(1)
        else:
            xor_result.append(0)
    return xor_result

def bitsAND(bits1, bits2):
    and_result = []

    for i in range(len(bits1)):
        if(bits1[i] == 1 and bits2[i] == 1):
            and_result.append(1)
        else:
            and_result.append(0)
    return and_result

def bitsNOT(bits):
    not_result = []
    for i in range(len(bits)):
        if(bits[i] == 0):
            not_result.append(1)
        else:
            not_result.append(0)
    return not_result

def bitsADD(bits1, bits2):
    max_len = max(len(bits1), len(bits2))
    
    if(len(bits1) > len(bits2)):
        bits2 = [0] * (len(bits1)- len(bits2)) + bits2
    if(len(bits2) > len(bits1)):
        bits1 = [0] * (len(bits2)- len(bits1)) + bits1
          
    result = []
      
    carry = 0
      
    for i in range(max_len - 1, -1, -1):
        r = carry
        r += 1 if bits1[i] == 1 else 0
        r += 1 if bits2[i] == 1 else 0
        result.insert(0, 1 if r % 2 == 1 else 0)
      
        carry = 0 if r < 2 else 1
      
    if(carry != 0):
        result.insert(0, 1)

    return result[-max_len:]

def bitsADDs(bits_list):
    added = bitsADD(bits_list[0], bits_list[1])
    if(len(bits_list) > 2):
        for i in range(2, len(bits_list)):
            added = bitsADD(added, bits_list[i])
    return added[-len(bits_list[0]):]
    

In [9]:
def SHA256(message):
    
    h0 = initial_h0
    h1 = initial_h1
    h2 = initial_h2
    h3 = initial_h3
    h4 = initial_h4
    h5 = initial_h5
    h6 = initial_h6
    h7 = initial_h7
    
    k = initial_k
    
    chunks = preprocessing(message)
    for chunk in chunks:
        w = create_message_schedule(chunk)

        a = h0
        b = h1
        c = h2
        d = h3
        e = h4
        f = h5
        g = h6
        h = h7
        
        for i in range(64):
            
            printx(k[i], 'k[i]')
            printx(w[i], 'w[i]')
            
            S1      = bitsXOR(bitsXOR(rightrotate(e, 6), rightrotate(e, 11)), rightrotate(e, 25))
            ch      = bitsXOR(bitsAND(e, f), bitsAND(bitsNOT(e), g))
            temp1   = bitsADDs([h, S1, ch, k[i], w[i]])
            S0      = bitsXOR(bitsXOR(rightrotate(a, 2), rightrotate(a, 13)), rightrotate(a, 22))
            maj     = bitsXOR(bitsXOR(bitsAND(a, b), bitsAND(a, c)), bitsAND(b, c))
            temp2   = bitsADDs([S0, maj])
                
            
            h = g
            g = f
            f = e
            e = bitsADD(d, temp1)
            d = c
            c = b
            b = a
            a = bitsADD(temp1, temp2)
            
            printx(S1, "S1")
            printx(ch, "ch")
            printx(temp1, "temp1")
            printx(S0, "S0")
            printx(temp2, "temp2")
            
            printx(a, 'a')
            printx(b, 'b')
            printx(c, 'c')
            printx(d, 'd')
            printx(e, 'e')
            printx(f, 'f')
            printx(g, 'g')
            printx(h, 'h')
            print("======================================")
            print()
            
            
        h0 = bitsADD(h0, a)
        h1 = bitsADD(h1, b)
        h2 = bitsADD(h2, c)
        h3 = bitsADD(h3, d)
        h4 = bitsADD(h4, e)
        h5 = bitsADD(h5, f)
        h6 = bitsADD(h6, g)
        h7 = bitsADD(h7, h)
        
    digest = bits2hex(h0) + bits2hex(h1) + bits2hex(h2) + bits2hex(h3) + bits2hex(h4) + bits2hex(h5) + bits2hex(h6) + bits2hex(h7)

    return digest

print(SHA256('hello world'))


==> k[i] <==
length list : 32
[0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0]


==> w[i] <==
length list : 32
[0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0]


==> S1 <==
length list : 32
[0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]


==> ch <==
length list : 32
[0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0]


==> temp1 <==
length list : 32
[0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0]


==> S0 <==
length list : 32
[1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]


==> temp2 <==
length list : 32
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1]


==> a <==
length list : 32
[0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 